### Постановка задачи

Просмотр фильмов на оригинальном языке - это популярный и действенный метод обучения иностранным языкам. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал бы 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

Наша задача - разработать ML решение для автоматического определения уровня сложности англоязычных фильмов. 

Программа-максимум:

           - языковая модель, 
           - веб-интерфейс,
           - микросервис.  
           
### Знакомство с исходными данными

В качестве исходных данных нам предоставлены размеченные по уровню сложности фильмы и субтитры к ним, словари английского языка с набором слов по уровню.

Сначала взглянем на эти файлы, чтоб понять, как можно было бы с ними вообще работать.

In [1]:
import pysrt

In [2]:
path_A2 = './English_level/English scores/Subtitles_all/A2/The Walking Dead-S01E01-Days Gone Bye.English.srt'

In [46]:
subs = pysrt.open(path_A2)

In [47]:
len(subs)

673

In [48]:
for i in range(0, len(subs), 50):
    print(subs[i])

1
00:00:03,169 --> 00:00:05,171
( bugs chittering )

51
00:06:24,376 --> 00:06:27,045
- And what do you say to that?
- I know what I want to say.

101
00:09:19,207 --> 00:09:22,085 X1:0
Dispatcher:
Unit 1, unit 3, to eastbound Route 18,

151
00:12:53,996 --> 00:12:56,332
Shh shh. That's it. Do you hear me?
Shh shh shh. Okay.

201
00:20:23,438 --> 00:20:25,440
( bugs chittering )

251
00:28:20,153 --> 00:28:23,030
Take a moment,
look how sharp it is.

301
00:31:39,304 --> 00:31:40,388
Yeah.

351
00:34:22,709 --> 00:34:26,086
Man: Don't look.
Get away from the windows.

401
00:38:37,536 --> 00:38:39,705
That's right.

451
00:41:46,000 --> 00:41:47,835
Conserve your ammo.

501
00:47:57,069 --> 00:47:59,071
( growling )

551
00:52:00,594 --> 00:52:02,429
You're all right.
Go on. You're all right.

601
00:57:15,775 --> 00:57:17,485
Good boy.

651
01:05:33,926 --> 01:05:36,595
♪ Of the place I know so well ♪



Чтобы проанализировать слова, нужно из этой структуры убрать тайминг, нумерацию кадров (видимо, это строки, потому что их количество равно длине файла с субтитрами), убрать слова в скобках, потому что в задаче запрос на понимание диалогов, а в скобках не диалоги, убрать знаки препинания и цифры, привести к нижнему регистру.

### Создание единого датасета из предложенных данных

In [6]:
#import pyprind
import pandas as pd
import os


basepath = './English_level/English scores/Subtitles_all/'

df_1 = pd.DataFrame(columns = ['Movie', 'Level', 'Srt'])


for s in ('A2', 'B1', 'B2', 'C1'):
    
    path = os.path.join(basepath, s)
    
    for file in os.listdir(path):
        
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
            
        df_1.loc[len(df_1)] = [file[:-4], s, txt]
        
#df_1.columns = ['Movie', 'Level', 'Srt']

In [7]:
display(df_1)

,Movie,Level,Srt
0,The Walking Dead-S01E01-Days Gone Bye.English,A2,"1\n00:00:03,169 --> 00:00:05,171\n( bugs chitt..."
1,The Walking Dead-S01E02-Guts.English,A2,"1\n00:00:03,045 --> 00:00:05,047\n- ( birds ch..."
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,A2,"1\n00:00:03,003 --> 00:00:04,671\n( thunder ru..."
3,The Walking Dead-S01E04-Vatos.English,A2,"1\n00:00:03,045 --> 00:00:05,422\n( birds chir..."
4,The Walking Dead-S01E05-Wildfire.English,A2,"1\n00:00:03,420 --> 00:00:04,922\n- ( walkie-t..."
...,...,...,...
158,Suits.S03E06.720p.HDTV.x264-mSD,C1,"ï»¿1\n00:00:01,383 --> 00:00:02,751\nI lost Av..."
159,Suits.S03E07.HDTV.x264-mSD,C1,"ï»¿1\n00:00:00,052 --> 00:00:01,352\nPreviousl..."
160,Suits.S03E08.480p.HDTV.x264-mSD,C1,"ï»¿1\n00:00:01,436 --> 00:00:03,028\nI get Ava..."
161,Suits.S03E09.480p.HDTV.x264-mSD,C1,"ï»¿1\n00:00:00,024 --> 00:00:01,478\nPreviousl..."


В первой таблице, собранной из данных папки заказчика с распределением субтитров по уровню языка,  у нас есть полный набор данных - название фильма, его рейтинг, текст субтитров.

In [8]:
#pbar = pyprind.ProgBar(50000)
df_2 = pd.DataFrame(columns = ['Movie', 'Srt'])
path = os.path.join(basepath, 'Subtitles')
for file in os.listdir(path):
    if file!='.DS_Store':
        #print(file)
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
        df_2.loc[len(df_2)] = [file[:-4], txt]
        #pbar.update()
#df_2.columns = ['Movie', 'Srt']

In [9]:
display(df_2)

,Movie,Srt
0,10_Cloverfield_lane(2016),"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,Aladdin(1992),"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
3,All_dogs_go_to_heaven(1989),"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
4,An_American_tail(1986),"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
...,...,...
110,Warm_bodies(2013),"2\n00:00:26,559 --> 00:00:28,627\n<i>What am I..."
111,Westworld_scenes_of_Dr_Robert_Ford,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2..."
112,We_are_the_Millers(2013),"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go..."
113,While_You_Were_Sleeping(1995),"1\n00:02:20,760 --> 00:02:24,720\nLUCY: <i>Oka..."


Во второй таблице, тоже собранной из данных файлов в папке заказчика, но другой, не размеченной по уровню языка, есть названия фильмов и текст субтитров.

In [10]:
df_3 = pd.read_excel('./English_level/English scores/movies_labels.xlsx')

In [11]:
display(df_3)

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


In [12]:
df_3.drop(columns='id', axis=1)

,Movie,Level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
...,...,...
236,Matilda(2022),C1
237,Bullet train,B1
238,Thor: love and thunder,B2
239,Lightyear,B2


В третьей таблице, предоставленной самим заказчиком, есть названия фильмов и уровень языка.

Теперь нам предстоит объединить эти данные в одну таблицу.

Столбцом для объединения точно будет название фильма. Объединение df_3 с df_2 не составит труда.

In [13]:
df_2_3 = df_3.merge(df_2, on='Movie', how='outer').drop(columns='id')

In [14]:
len(df_2_3)

250

In [15]:
len(df_2)

115

In [16]:
len(df_3)

241

In [17]:
df_2_3.isna().sum()

Movie      0
Level      9
Srt      131
dtype: int64

In [18]:
df_2_3.sample(10)

,Movie,Level,Srt
125,Seven.Worlds.One.Planet.S01E02.2160p.BluRay.Re...,B1,NaN
140,Crazy4TV.com - Suits.S06E05.720p.BluRay.x265.H...,B2,NaN
169,Suits.Episode 7- Hitting Home,B2,NaN
14,Before_sunset(2004),"B1, B2","1\n00:00:05,000 --> 00:00:15,000\nCreated and ..."
186,Suits.S02E03.HDTV.x264-ASAP,B2,NaN
67,Pirates_of_the_Caribbean(2003),B1,"1\n00:00:29,409 --> 00:00:33,697\n<i>Drink up,..."
109,We_are_the_Millers(2013),B1,"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go..."
137,Crazy4TV.com - Suits.S06E02.720p.BluRay.x265.H...,B2,NaN
38,Powder(1995),B1,"1\n00:01:19,208 --> 00:01:20,333\nEMT 1:\nAll ..."
209,Suits S04E01 EngSub,C1,NaN


In [19]:
df_all = df_2_3.merge(df_1, on='Movie', how='outer')

In [51]:
df_all.sample(4)

,Movie,Level_x,Srt_x,Level_y,Srt_y,Level,Srt
227,Suits.S03E03.480pHDTV.x264-mSD,C1,NaN,C1,"ï»¿1\n00:00:00,061 --> 00:00:01,605\nPreviousl...",C1,"ï»¿1\n00:00:00,061 --> 00:00:01,605\nPreviousl..."
106,Up (2009),A2,NaN,NaN,NaN,A2,NaN
280,Virgin.River.S01E01.INTERNAL.720p.WEB.x264-STRiFE,NaN,NaN,B2,"1\n00:00:23,982 --> 00:00:26,362\nâª I woke u...",B2,"1\n00:00:23,982 --> 00:00:26,362\nâª I woke u..."
52,Logan(2017),B1,"1\n00:00:58,157 --> 00:00:59,715\nMAN 1: We ge...",NaN,NaN,B1,"1\n00:00:58,157 --> 00:00:59,715\nMAN 1: We ge..."


In [21]:
df_all.isna().sum()

Movie        0
Level_x     49
Srt_x      171
Level_y    127
Srt_y      127
dtype: int64

In [22]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
dtypes: object(5)
memory usage: 11.5+ KB


In [23]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2', nan],
      dtype=object)

In [24]:
df_all['Level_y'].unique()

array([nan, 'A2', 'B1', 'B2', 'C1'], dtype=object)

In [25]:
df_all['Level_x'] = df_all['Level_x'].replace('A2/A2+', 'A2')
df_all['Level_x'] = df_all['Level_x'].replace('A2/A2+, B1', 'B1')

In [26]:
df_all['Level_x'] = df_all['Level_x'].replace('B1, B2', 'B2')

In [27]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2', 'C1', nan], dtype=object)

In [28]:
print('Строк с разными значениями в столбцах Label_x и Label_y:',\
    len(df_all.loc[(df_all['Level_x'].notna())&(df_all['Level_y'].notna()) & (df_all['Level_x']!=df_all['Level_y'])]))

Строк с разными значениями в столбцах Label_x и Label_y: 0


Значит, просто заполним пропуски переносом значений из одного стобца в другой.

In [29]:
df_all['Level']=df_all['Level_x'].fillna(df_all['Level_y'])

In [30]:
df_all['Level'].unique()

array(['B1', 'B2', 'A2', 'C1', nan], dtype=object)

In [31]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    281 non-null    object
dtypes: object(6)
memory usage: 13.7+ KB


Также проверим, есть ли несовпадения по субтитрам.

In [33]:
print('Строк с разными значениями в столбцах Srt_x и Srt_y:',\
    len(df_all.loc[(df_all['Srt_x'].notna())&(df_all['Srt_y'].notna()) & (df_all['Srt_x']!=df_all['Srt_y'])]))

Строк с разными значениями в столбцах Srt_x и Srt_y: 0


In [34]:
df_all['Srt']=df_all['Srt_x'].fillna(df_all['Srt_y'])

In [35]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    281 non-null    object
 6   Srt      282 non-null    object
dtypes: object(7)
memory usage: 16.0+ KB


In [36]:
df_all.columns

Index(['Movie', 'Level_x', 'Srt_x', 'Level_y', 'Srt_y', 'Level', 'Srt'], dtype='object')

In [37]:
df = df_all.copy().drop(columns=['Level_x', 'Srt_x', 'Level_y', 'Srt_y'])

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   290 non-null    object
 1   Level   281 non-null    object
 2   Srt     282 non-null    object
dtypes: object(3)
memory usage: 6.9+ KB


Если пропуски в разных строках, то удалится в общей сложности 17 строк. Пока восстанавливать пропущенные значения не буду - надо смочь выполнить работу в минимальном объёме.

In [39]:
df = df.dropna(subset=['Level', 'Srt'])

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 289
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   273 non-null    object
 1   Level   273 non-null    object
 2   Srt     273 non-null    object
dtypes: object(3)
memory usage: 8.5+ KB


Теперь посмотрим на дубли. Найдём - удалим.

In [41]:
df.duplicated().sum()

3

In [42]:
df.drop_duplicates()

,Movie,Level,Srt
0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,Aladdin(1992),A2,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
...,...,...,...
285,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:18,852 --> 00:00:19,852\nHey.\n\n2\n0..."
286,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:10,468 --> 00:00:13,178\nAre you sure..."
287,Virgin.River.S01E08.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:07,382 --> 00:00:10,012\nTwo IVs in p..."
288,Virgin.River.S01E09.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:16,474 --> 00:00:18,024\nOh.\n\n2\n00..."


In [43]:
import numpy as np

np.random.seed(123)
df = df.reindex(np.random.permutation(df.index)).reset_index()

In [44]:
#При сохранении в формате csv теряется часть строк субтитров, видимо, из-за служебных символов.
#Пока не решила эту проблему, не сохраняю в файл.

#df.to_csv('./master2/movie_data.csv', index=False)

#df = pd.read_csv('./master2/movie_data.csv')
#df.head(8)

In [45]:
df.head(8)

,index,Movie,Level,Srt
0,169,Suits.Episode 7- Hitting Home,B2,"ï»¿1\n00:00:07,364 --> 00:00:09,093\n<i>You wa..."
1,221,Suits S04E13 EngSub,C1,"ÿþ1 \n \n 0 0 : 0 0 : 0 1 , 8 2 6   - - >   0 ..."
2,130,Seven.Worlds.One.Planet.S01E07.2160p.BluRay.Re...,B1,"ï»¿1\n00:00:50,176 --> 00:00:51,677\nAfrica.\n..."
3,260,"Crown, The S01E05 - Smoke and Mirrors.en.FORCED",B2,"ï»¿1\n00:41:36,960 --> 00:41:39,960\n<i>For th..."
4,219,Suits S04E11 EngSub,C1,"ÿþ1 \n \n 0 0 : 0 0 : 0 2 , 3 2 6   - - >   0 ..."
5,19,Cast_away(2000),A2,"1\n00:00:05,000 --> 00:00:15,000\nCreated and ..."
6,224,Suits S04E16 EngSub,C1,"ÿþ1 \n \n 0 0 : 0 0 : 0 1 , 7 7 0   - - >   0 ..."
7,80,The_Intern(2015),B2,"1\n00:01:11,418 --> 00:01:14,887\nFreud said, ..."


In [ ]:
df.drop('index', axis=1)

In [ ]:
del df_1, df_2, df_3, df_2_3, df_all

Удалила уже ненужные переменные.

### Обработка субтитров

Необходимо преобразовать текст в числовую форму, удобную для моделирования.

Сначала избавимся от лишних символов, стоп-слов и скобок.